# The Revenge of Rocchio's Angels

We will be #1 this time

Local Script Dependencies

In [1]:
from engine import SearchEngine
from evaluate_map import *
from optimizing import Optimize

D:\PycharmEnvs\FinalProject\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


Full (Current) Pipeline

In [2]:
se = SearchEngine()

In [3]:
se.set_searcher(fb_terms=20, fb_docs=5, original_query_weight=0.6, mu=300) #MAP=0.264891

In [4]:
topics = load_topics("Data/queriesROBUST.txt")

In [5]:
se.search_all_queries(topics, k=1000, m=100, output_file="Results/hey05.txt", fusion_weight=0.5)

Searching topics:   0%|          | 1/249 [00:01<07:37,  1.84s/it]D:\PycharmProjects\FinalProject\engine.py:45: RuntimeWarning: invalid value encountered in scalar divide
  return max(scores)*max_weight + (1-max_weight)*(sum(scores)-max(scores))/(len(scores)-1)
Searching topics: 100%|██████████| 249/249 [04:53<00:00,  1.18s/it]


In [7]:
qrels = load_qrels("Data/qrels_50_Queries")   # or "qrel301.txt"
run   = load_run("Results/hey05.txt")

map_score, ap_by_q = mean_average_precision(qrels, run)
map_score

0.2844271042048603

 Extract Train Set Results

In [ ]:
HARD_QUERIES =[309, 308, 338, 344, 348, 320, 328, 334, 303, 339] # From EDA

In [ ]:
all_hits = {}
hard_hits = {}
for i, (qid, topic) in enumerate(topics.items()):
    results = se.get_top_k(topic, k=1000, clean=True)
    all_hits[f"{qid}_{topic}"] = results
    if int(qid) in HARD_QUERIES:
        hard_hits[f"{qid}_{topic}"] = results
    if i==49:
        print(qid)
        break

import pickle
with open("pkls/top1000_rm3_train.pkl", "wb") as f:
    pickle.dump(all_hits, f)
with open("pkls/top1000_rm3_train_hard.pkl", "wb") as f:
    pickle.dump(hard_hits, f)

In [ ]:
res = se.retrieve_rerank("international organized crime", k=1000, m=50)

In [ ]:
res

In [ ]:
# from pyserini.analysis import Analyzer, get_lucene_analyzer
# analyzer = get_lucene_analyzer(stemmer='porter', stopwords=False)
# se.reader.get_term_counts("spanish",analyzer) #(df,cf)

In [ ]:
# This one creates a file called "run.txt" with submitting format, can change file name
se.search_all_queries(topics, k=5, m=2, output_file="Results/hey.txt")

In [ ]:
qrels = load_qrels("Data/qrels_50_Queries")   # or "qrel301.txt"
run   = load_run("Results/run.txt")

map_score, ap_by_q = mean_average_precision(qrels, run)
map_score

In [ ]:
stats = se.reader.stats()
print(f"average terms per doc: {stats['total_terms']/stats['documents']}")

In [ ]:
doc = se.searcher.doc("FT921-3160")
text = doc.raw()
text

In [ ]:
map = get_map_by_paths("Data/qrels_50_Queries", "Results/run.txt")

In [ ]:
opti = Optimize()
# fb_terms_values = [5, 6, 8, 10, 15, 20]
# fb_docs_values = [5, 7, 10, 15]
# og_query_weight_values = [0.3, 0.4, 0.5, 0.6, 0.7]
mus = [200,300,400,500,600,700,800,900,1000,1100,1200]

opti.optimize_qld(topics, [20], [5], [0.6], mus, k=1000)